In [9]:
import sys
import numpy as np
import cv2
from math import pi, atan, cos, sin, acos, sqrt, tan
from scipy.interpolate import RectBivariateSpline

In [10]:
def projection_angle(x, d): 
     x_max = (1 + d) / d 
     numerator = -2 * d * x ** 2 + 2 * (d + 1) * sqrt((1 - d ** 2) * x ** 2 + (d + 1) ** 2)
     denominator = 2 * (x ** 2 + (d + 1) ** 2)

     if 0 < x < x_max:
        project_angle = acos(numerator / denominator)
     elif x < 0:
        project_angle = - acos(numerator / denominator)
     elif x == x_max:
        project_angle = pi/2.
     else:
        raise Exception('invalid input args')

     return project_angle

In [11]:
def panotostereo(panorama, distance):

    frames = []
    input_img = panorama
    height, width, _ = input_img.shape
    d = distance
    xp_max = (1 + d) / d  
    yp_max = (1 + d) / d  
    xp_domain = xp_max * (np.arange(-1., 1., 2. / height) + 1.0 / height)
    yp_domain = yp_max * (np.arange(-1., 1., 2. / height) + 1.0 / height)
    """Get the radian values of each pixel in the image with delta_rad"""
    delta_rad = 2 * pi / width  
    
    for face in range(4):
        print('generating stereo image', face)
        output_img = np.zeros((height, height, 3))
        
        """Use scipy's interpolation function"""
        interpolate_0 = RectBivariateSpline(np.arange(height), np.arange(width), input_img[:, :, 0])
        interpolate_1 = RectBivariateSpline(np.arange(height), np.arange(width), input_img[:, :, 1])
        interpolate_2 = RectBivariateSpline(np.arange(height), np.arange(width), input_img[:, :, 2])
        pano_x = np.zeros((height, 1))
        pano_y = np.zeros((height, 1))

        for j, xp in enumerate(xp_domain):
            phi = projection_angle(xp, d)
            pano_x[j] = (width / 2.0 + (phi / delta_rad))

        for i, yp in enumerate(yp_domain):
            theta = projection_angle(yp, d)
            pano_y[i] = height/2.0 + (theta/delta_rad)

        output_img[:, :, 0] = interpolate_0(pano_y, pano_x)
        output_img[:, :, 1] = interpolate_1(pano_y, pano_x)
        output_img[:, :, 2] = interpolate_2(pano_y, pano_x)

        cv2.imwrite('split_'+str(face)+'_'+str(d)+'.jpg', output_img)
        frames.append(output_img)
        input_img = np.concatenate(
            (input_img[:, int(width/4):, :], input_img[:, :int(width/4), :]), axis=1)
    return frames

In [35]:
im = cv2.imread('input/7frLY.jpg')  
projection = panotostereo(im, 2) 


generating stereo image 0
generating stereo image 1
generating stereo image 2
generating stereo image 3
